
<br>
Robust linear estimator fitting<br>
===============================<br>
Here a sine function is fit with a polynomial of order 3, for values<br>
close to zero.<br>
Robust fitting is demoed in different situations:<br>
- No measurement errors, only modelling errors (fitting a sine with a<br>
  polynomial)<br>
- Measurement errors in X<br>
- Measurement errors in y<br>
The median absolute deviation to non corrupt new data is used to judge<br>
the quality of the prediction.<br>
What we can see that:<br>
- RANSAC is good for strong outliers in the y direction<br>
- TheilSen is good for small outliers, both in direction X and y, but has<br>
  a break point above which it performs worse than OLS.<br>
- The scores of HuberRegressor may not be compared directly to both TheilSen<br>
  and RANSAC because it does not attempt to completely filter the outliers<br>
  but lessen their effect.<br>


In [ ]:
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
from sklearn.linear_model import (
    LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor)
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [ ]:
np.random.seed(42)

In [ ]:
X = np.random.normal(size=400)
y = np.sin(X)
# Make sure that it X is 2D
X = X[:, np.newaxis]

In [ ]:
X_test = np.random.normal(size=200)
y_test = np.sin(X_test)
X_test = X_test[:, np.newaxis]

In [ ]:
y_errors = y.copy()
y_errors[::3] = 3

In [ ]:
X_errors = X.copy()
X_errors[::3] = 3

In [ ]:
y_errors_large = y.copy()
y_errors_large[::3] = 10

In [ ]:
X_errors_large = X.copy()
X_errors_large[::3] = 10

In [ ]:
estimators = [('OLS', LinearRegression()),
              ('Theil-Sen', TheilSenRegressor(random_state=42)),
              ('RANSAC', RANSACRegressor(random_state=42)),
              ('HuberRegressor', HuberRegressor())]
colors = {'OLS': 'turquoise', 'Theil-Sen': 'gold', 'RANSAC': 'lightgreen', 'HuberRegressor': 'black'}
linestyle = {'OLS': '-', 'Theil-Sen': '-.', 'RANSAC': '--', 'HuberRegressor': '--'}
lw = 3

In [ ]:
x_plot = np.linspace(X.min(), X.max())
for title, this_X, this_y in [
        ('Modeling Errors Only', X, y),
        ('Corrupt X, Small Deviants', X_errors, y),
        ('Corrupt y, Small Deviants', X, y_errors),
        ('Corrupt X, Large Deviants', X_errors_large, y),
        ('Corrupt y, Large Deviants', X, y_errors_large)]:
    plt.figure(figsize=(5, 4))
    plt.plot(this_X[:, 0], this_y, 'b+')
    for name, estimator in estimators:
        model = make_pipeline(PolynomialFeatures(3), estimator)
        model.fit(this_X, this_y)
        mse = mean_squared_error(model.predict(X_test), y_test)
        y_plot = model.predict(x_plot[:, np.newaxis])
        plt.plot(x_plot, y_plot, color=colors[name], linestyle=linestyle[name],
                 linewidth=lw, label='%s: error = %.3f' % (name, mse))
    legend_title = 'Error of Mean\nAbsolute Deviation\nto Non-corrupt Data'
    legend = plt.legend(loc='upper right', frameon=False, title=legend_title,
                        prop=dict(size='x-small'))
    plt.xlim(-4, 10.2)
    plt.ylim(-2, 10.2)
    plt.title(title)
plt.show()